In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

In [2]:
from siamnet import api

In [3]:
save_path = '../data_omniglot/'

In [4]:
with open(os.path.join(save_path,"train.pickle"), "rb") as file:
    imgs_train = pickle.load(file)

In [5]:
with open(os.path.join(save_path,"val.pickle"), "rb") as file:
    imgs_validate = pickle.load(file)

In [6]:
tf.reset_default_graph()

In [7]:
img1 = tf.placeholder(tf.float32, shape=[None, 105, 105, 1])
img2 = tf.placeholder(tf.float32, shape=[None, 105, 105, 1])
diffclass = tf.placeholder(tf.float32, shape=[None,1])

In [8]:
m = 1

features1 = api.convnet(img1, reuse=False)
features2 = api.convnet(img2, reuse=True)

diff = tf.sqrt(tf.reduce_mean(tf.square(features1 - features2),axis=1))
diff = tf.reshape(diff,[-1,1])

loss = tf.reduce_mean((1-diffclass)*diff**2 / 2 + diffclass*(tf.maximum(0.,m-diff))**2/2)

opt = tf.train.AdamOptimizer(
        learning_rate = 0.0001)
train_op = opt.minimize(loss)

In [9]:
saver = tf.train.Saver()

In [ ]:
batchsize = 64
trainsteps = 5
printstep = 500
l = np.zeros(trainsteps)
acc = np.zeros(trainsteps)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(trainsteps):
        batch, batch_diff = api.getbatch(imgs_train, batchsize)
        _,l[step] = sess.run([train_op,loss], feed_dict={img1: batch[0,:,:,:,:],
                                               img2: batch[1,:,:,:,:],
                                               diffclass: batch_diff})
        if ((step+1)%printstep==0):
            print('Step %d: Loss %f, Accuracy %f' % (step,np.mean(l[step-printstep+1:step+1]),
                                                     np.mean(acc[step-printstep+1:step+1])))
            
    saver.save(sess, './checkpoints/model', step)
#         print(np.matrix([p[:,0],batch_diff[:,0]]))

In [ ]:
plt.plot(l)

In [ ]:
ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/model'))

## compare image 0 to images 15-19 and find closest match

In [ ]:
num_compare = 5

In [ ]:
find_img = 127

In [ ]:
plt.imshow(imgs_validate[find_img,0,:,:])
plt.show()

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, ckpt.model_checkpoint_path)
    diff_val=np.zeros([imgs_validate.shape[0],num_compare])
    for i in range(imgs_validate.shape[0]):
        for j in range(20-num_compare,20):
            diff_val[i][j-20+num_compare] = sess.run(diff, feed_dict={
                img1: np.reshape(imgs_validate[find_img,0,:,:],[1,105,105,1]),
                img2: np.reshape(imgs_validate[i,j,:,:],[1,105,105,1])})

In [ ]:
closest_match = np.argmin(np.mean(diff_val,1))
print(closest_match)

In [ ]:
print(diff_val[find_img])
print(diff_val[closest_match])

In [ ]:
fig, ax = plt.subplots(3,num_compare,figsize=(20,10))
ax[0,0].imshow(imgs_validate[find_img,0,:,:])
for i in range(20-num_compare,20):
    ax[1,i-20+num_compare].imshow(imgs_validate[find_img,i,:,:])
    ax[2,i-20+num_compare].imshow(imgs_validate[closest_match,i,:,:])  
plt.show()